In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
class StdScalerByGroup(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # X may not be a pandas dataframe (e.g. a np.array)
        df = pd.DataFrame(X)
        
        # A dictionary of means/standard-deviations for each column, for each group.
        dict_out = {}
        for i in set(df[df.columns[0]]):
            dict_mid = {}
            temp = df[df[df. columns[0]] == i]
            for j in (temp.columns[1:]):
                dict_in = {"mean": temp[j].mean(), "std": temp.std()[j]}
                dict_mid[j] = dict_in
            dict_out[i] = dict_mid
        self.grps_ = dict_out

        return self

    def transform(self, X, y=None):
        try:
            getattr(self, "grps_")
        except AttributeError:
            raise RuntimeError("You must fit the transformer before tranforming the data!")
        
        # Define a helper function here?
        def hell0(a,b, col):
            return (b - self.grps_[a][col]["mean"]) / self.grps_[a][col]["std"]
        # X may not be a dataframe (e.g. np.array)

        df = pd.DataFrame(X)
        res = pd.DataFrame()
        for i in df.columns[1:]:
            res[i] = list(map(hell0, np.array(df[df.columns[0]]), np.array(df[i]), [i] * len(df[i])))
        return res

In [3]:
df = pd.read_csv("train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df = pd.read_csv("train.csv")
df["Name"] = df.Name.apply(lambda x: x.split(", ")[1].split(".")[0] if x is not np.NaN else "NAN")
df["Age"] = df.Age.fillna(df.Age.mean())
df["Age"] = df["Age"].apply(lambda x: int(x))
df["Cabin"] = df.Cabin.fillna("NAN")
df.head()

df_men = df[df.Sex == "male"]
x_train_men = df_men.drop("PassengerId Sex Survived Ticket".split(), axis = 1)
y_train_men = df_men.Survived

df_women = df[df.Sex == "female"]
x_train_women = df_women.drop("PassengerId Sex Survived Ticket".split(), axis = 1)
y_train_women = df_women.Survived

In [5]:
pl_men = Pipeline([('col_trans', ColumnTransformer([("age-stdScaler", StdScalerByGroup(), ["Pclass", "Age"]), ("sex-one-hot", OneHotEncoder(drop = 'first', handle_unknown= 'ignore'), ["Name", "Cabin", "Embarked"])], remainder = "passthrough")), ('DTC', DecisionTreeClassifier(max_depth = 3))])
pl_women = Pipeline([('col_trans', ColumnTransformer([("age-stdScaler", StdScalerByGroup(), ["Pclass", "Age"]), ("sex-one-hot", OneHotEncoder(drop = 'first', handle_unknown= 'ignore'), ["Name", "Cabin", "Embarked"])], remainder = "passthrough")), ('DTC', DecisionTreeClassifier(max_depth = 3))])

In [6]:
pl_men.fit(x_train_men, y_train_men)
pl_men.score(x_train_men, y_train_men)

0.8509532062391681

In [7]:
pl_women.fit(x_train_women, y_train_women)
pl_women.score(x_train_women, y_train_women)

0.8152866242038217

In [8]:
df_test = pd.read_csv("test.csv")
df_test["Name"] = df_test.Name.apply(lambda x: x.split(", ")[1].split(".")[0] if x is not np.NaN else "NAN")
df_test["Age"] = df_test.Age.fillna(df_test.Age.mean())
df_test["Age"] = df_test["Age"].apply(lambda x: int(x))
df_test["Cabin"] = df_test.Cabin.fillna("NAN")
df_test["Fare"] = df_test.Fare.fillna(df_test.Fare.mean())

In [9]:
df_test_men = df_test[df_test.Sex == "male"]
x_test_men = df_test_men.drop("PassengerId Sex Ticket".split(), axis = 1)

df_test_women = df_test[df_test.Sex == "female"]
x_test_women = df_test_women.drop("PassengerId Sex Ticket".split(), axis = 1)

In [10]:
df_test_men.PassengerId

0       892
2       894
3       895
5       897
7       899
       ... 
407    1299
413    1305
415    1307
416    1308
417    1309
Name: PassengerId, Length: 266, dtype: int64

In [12]:
res = pd.DataFrame()
res["PassengerId"] = df_test.PassengerId
res["Survived"] = [men_sur[df_test.PassengerId.iloc[i]] if df_test.Sex.iloc[i] == 'male' else women_sur[df_test.PassengerId.iloc[i]] for i in range(df_test.shape[0])]
res.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [13]:
res.set_index("PassengerId").to_csv("res.csv")